# Running BigQuery ML

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
sb.set()


### The Data

In [3]:
%%bigquery earnings_data
SELECT *
FROM `crazy-hippo-01.clv.earnings_per_year`

In [4]:
earnings_data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,Private,297847,9th,5,Married-civ-spouse,Other-service,Wife,Black,Female,3411,0,34,United-States,<=50K
1,72,Private,74141,9th,5,Married-civ-spouse,Exec-managerial,Wife,Asian-Pac-Islander,Female,0,0,48,United-States,>50K
2,45,Private,178215,9th,5,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0,0,40,United-States,>50K
3,31,Private,86958,9th,5,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
4,55,Private,176012,9th,5,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,23,United-States,<=50K


### Training a model in BigQuery ML

#### Using Logistic Regression for this example

In [2]:
%%bigquery 
CREATE OR REPLACE MODEL clv.earnings_model
OPTIONS(input_label_cols=['income'], model_type='logistic_reg')
AS
SELECT *
FROM
  `crazy-hippo-01.clv.earnings_per_year`

""


#### The <b>ML.FEATURE_INFO</b> function allows you to see information about the input features used to train a model.

In [6]:
%%bigquery 
SELECT * 
FROM ML.FEATURE_INFO(MODEL `crazy-hippo-01.clv.earnings_model`)

,input,min,max,mean,median,stddev,category_count,null_count
0,age,17.0,90.0,38.580010,37.0,13.667871,NaN,0.0
1,workclass,NaN,NaN,NaN,NaN,NaN,9.0,NaN
2,fnlwgt,13769.0,1484705.0,189622.921468,177951.0,105574.934549,NaN,0.0
3,education,NaN,NaN,NaN,NaN,NaN,16.0,NaN
4,education_num,1.0,16.0,10.074694,10.0,2.574967,NaN,0.0
5,marital_status,NaN,NaN,NaN,NaN,NaN,7.0,NaN
6,occupation,NaN,NaN,NaN,NaN,NaN,15.0,NaN
7,relationship,NaN,NaN,NaN,NaN,NaN,6.0,NaN
8,race,NaN,NaN,NaN,NaN,NaN,5.0,NaN
9,sex,NaN,NaN,NaN,NaN,NaN,2.0,NaN


#### Use the <b>ML.EVALUATE</b> function to evaluate model metrics. 

In [20]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `crazy-hippo-01.clv.earnings_model`)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.709845,0.540434,0.838483,0.613662,0.347407,0.8898


#### Use the <b>ML.CONFUSION_MATRIX</b> function to return a confusion matrix for the given logistic regression model and input data. 

In [24]:
%%bigquery
SELECT
  *
FROM
  ML.CONFUSION_MATRIX(MODEL `crazy-hippo-01.clv.earnings_model`,
  (
    SELECT
      *
    FROM
      `crazy-hippo-01.clv.earnings_per_year`))

,expected_label,___50K,__50K
0,<=50K,23053,1588
1,>50K,3522,4298


#### The <b>ML.WEIGHTS</b> function allows you to see the underlying weights used by a model during prediction.

In [22]:
%%bigquery
SELECT * 
FROM ML.WEIGHTS(MODEL `crazy-hippo-01.clv.earnings_model`
)

,processed_input,weight,category_weights
0,age,1.989957e-02,[]
1,workclass,NaN,"[{'category': ' Self-emp-inc', 'weight': 0.332..."
2,fnlwgt,3.201723e-07,[]
3,education,NaN,"[{'category': ' 7th-8th', 'weight': -0.7002128..."
4,education_num,1.239920e-01,[]
5,marital_status,NaN,"[{'category': ' Separated', 'weight': -0.60850..."
6,occupation,NaN,"[{'category': ' Adm-clerical', 'weight': -0.37..."
7,relationship,NaN,"[{'category': ' Unmarried', 'weight': -0.62860..."
8,race,NaN,"[{'category': ' Black', 'weight': -0.391409798..."
9,sex,NaN,"[{'category': ' Male', 'weight': -0.0538525515..."


## Batch Predictions 

#### Let us generate som samples to predit on

In [30]:
%%bigquery
CREATE or REPLACE TABLE `clv.prediction_sample`
AS (
SELECT *
FROM `crazy-hippo-01.clv.earnings_per_year`
WHERE RAND() < 0.0005 
)

""


#### Loading data from BQ and see how the data looks. 

In [34]:
%%bigquery prediction_data
SELECT * 
FROM `crazy-hippo-01.clv.prediction_sample`


In [35]:
prediction_data

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,27,Private,160786,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1902,40,United-States,>50K
1,46,Private,273629,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
2,36,Local-gov,61778,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,15024,0,40,United-States,>50K
3,54,Private,215990,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
4,43,Private,397280,Assoc-acdm,12,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,50,United-States,>50K
5,26,Private,103700,Some-college,10,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K
6,48,Private,232840,Some-college,10,Widowed,Adm-clerical,Unmarried,White,Female,0,0,43,United-States,<=50K
7,54,Private,240542,Some-college,10,Divorced,Sales,Unmarried,White,Female,0,0,48,United-States,<=50K
8,32,?,30499,Bachelors,13,Divorced,?,Unmarried,White,Female,0,0,32,United-States,<=50K
9,41,Private,106627,Assoc-acdm,12,Divorced,Exec-managerial,Unmarried,Black,Female,0,0,50,United-States,<=50K


#### Batch Predictions

In [33]:
%%bigquery
SELECT *
FROM
  ML.PREDICT(MODEL `crazy-hippo-01.clv.earnings_model`,
    (
    SELECT
      *
    FROM
      `crazy-hippo-01.clv.prediction_sample`))

,predicted_income,predicted_income_probs,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,<=50K,"[{'label': ' >50K', 'prob': 0.2975156380809634...",27,Private,160786,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1902,40,United-States,>50K
1,>50K,"[{'label': ' >50K', 'prob': 0.7964805186208006...",46,Private,273629,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
2,<=50K,"[{'label': ' >50K', 'prob': 0.4897753081576619...",36,Local-gov,61778,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,15024,0,40,United-States,>50K
3,>50K,"[{'label': ' >50K', 'prob': 0.501900596261103}...",54,Private,215990,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
4,<=50K,"[{'label': ' >50K', 'prob': 0.1638158349601476...",43,Private,397280,Assoc-acdm,12,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,50,United-States,>50K
5,<=50K,"[{'label': ' >50K', 'prob': 0.0372807235318270...",26,Private,103700,Some-college,10,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K
6,<=50K,"[{'label': ' >50K', 'prob': 0.0541007905844975...",48,Private,232840,Some-college,10,Widowed,Adm-clerical,Unmarried,White,Female,0,0,43,United-States,<=50K
7,<=50K,"[{'label': ' >50K', 'prob': 0.0906534809283018...",54,Private,240542,Some-college,10,Divorced,Sales,Unmarried,White,Female,0,0,48,United-States,<=50K
8,<=50K,"[{'label': ' >50K', 'prob': 0.0460866293422405...",32,?,30499,Bachelors,13,Divorced,?,Unmarried,White,Female,0,0,32,United-States,<=50K
9,<=50K,"[{'label': ' >50K', 'prob': 0.1259224686451649...",41,Private,106627,Assoc-acdm,12,Divorced,Exec-managerial,Unmarried,Black,Female,0,0,50,United-States,<=50K
